In [ ]:
plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
file_path="/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb"

In [ ]:
def extract_file_path(raw_data):
  n=len(raw_data)
  extensions=[b'.ipynb',b'.py']
  extpos=0
  asciirange=set(range(32,127))-set(ord(a) for a in '')
  end=0
  while end<n:
   end=0
   while end<n:
    delta=n-end
    extlen=0 
    for ext in extensions:
      pos=raw_data[end:end+delta].find(ext)
      if pos>0:
        extlen=len(ext)
        delta=pos
    if extlen==0 :
      break
    extpos=end+delta#position of the dot of the extension
    start=extpos
    end=extpos+extlen
    if extpos>=len(raw_data):
        break
#    print(f'{extpos = }, {chr(raw_data[extpos]) = }')
    while raw_data[start]in asciirange:
      start-=1
      if start<0:
        break
    start+=1
#    print(f'{start=} {end=} {end-start=} \n{raw_data[start-4:end+4]=}') 
   result.append(start,end)
  return raw_data.getitem()
def get_recent_doc_path(i=0):
  import plistlib, os
  return plistlib.load(open(os.environ["CFFIXED_USER_HOME"]+'/Library/Application Support/RecentFiles.plist','rb'))[i]
  
extract_file_path(get_recent_doc_path())

---------------------------------------------------------------------------NameError                                 Traceback (most recent call last)Cell In[11], line 37
     34   import plistlib, os
     35   return plistlib.load(open(os.environ["CFFIXED_USER_HOME"]+'/Library/Application Support/RecentFiles.plist','rb'))[i]
---> 37 extract_file_path(get_recent_doc_path())
Cell In[11], line 31, in extract_file_path(raw_data)
     29     start+=1
     30 #    print(f'{start=} {end=} {end-start=} \n{raw_data[start-4:end+4]=}') 
---> 31    result.append(start,end)
     32   return raw_data.getitem()
NameError: name 'result' is not defined

In [ ]:
import re,plistlib

# Define the allowed file extensions (customize as needed)
allowed_extensions = [b'txt', b'pdf', b'jpg', b'ipynb',b'py']
# Create the regex pattern for specific extensions
extension_pattern = b'|'.join(allowed_extensions)  # e.g., b'txt|pdf|jpg|ipynb
# Bytes pattern for Unix-style paths (absolute, relative, or plain filenames)
pattern = br'(?:(?:/)?[a-zA-Z0-9_\-\.\xC2-\xF4][a-zA-Z0-9_\-\.\x80-\xBF\xC2-\xF4]*(?:/[a-zA-Z0-9_\-\.\xC2-\xF4][a-zA-Z0-9_\-\.\x80-\xBF\xC2-\xF4]*)*)\.(?:' + extension_pattern + b')'

def extract_paths(binary_data):
    # Preprocess to replace invalid bytes with whitespace
    binary_data = binary_data.replace(b'\xA4', b' ')  # Replace \xA4 with space
    # Find all matches in the binary string
    try:
        matches = re.findall(pattern, binary_data)
    except re.error as e:
        print(f"Regex error: {e}")
        return []
    # Decode matches to strings, handling potential decoding errors
    paths = []
    for match in matches:
        try:
            # Attempt to decode as UTF-8
            path = match.decode('utf-8')
            paths.append(path)
        except UnicodeDecodeError:
            # Skip matches that can't be decoded (non-UTF-8 sequences)
            continue
    return paths
def get_current_notebook_path():
  import plistlib
  plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
  return extract_paths(plistlib.load(open(plist_path,'rb'))[0])[-1]
  
print(f"{get_current_notebook_path() = }\n")

# Example usage
print('Example for "extract_paths(binary_data)" function:\n')
binary_data = b"data /Users/test/file.txt\xA4invalid/home/\xe6\x96\x87\xe6\xa1\xa3/doc.pdf /var/log/test.doc PythonistalabTest.ipynb /private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb"
paths = extract_paths(binary_data)
print(paths)
# Expected Output: ['/Users/test/file.txt', 'invalid/home/文档/doc.pdf', 'PythonistalabTest.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb']
print("\n")
print("File paths in RecentFiles.plist:\n")
for binary_data in plistlib.load(open(plist_path,'rb')):
  paths = extract_paths(binary_data)
  print(paths)
  print()


get_current_notebook_path() = '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb'

Example for "extract_paths(binary_data)" function:

['/Users/test/file.txt', 'invalid/home/文档/doc.pdf', 'PythonistalabTest.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb']


File paths in RecentFiles.plist:

['PythonistalabTest.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb']

['get_path.py', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/get_path.py']

['pandoc.py', '/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Documents/Documents/site-packages/jupytext/pandoc.py']

['Trackpad.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4

In [ ]:
import re, plistlib

# Allowed file extensions (customizable)
allowed_extensions = [b'ipynb', b'py']
extension_pattern = b'|'.join(allowed_extensions)
pattern = br'(?:(?:/)?[a-zA-Z0-9_\-\.\xC2-\xF4][a-zA-Z0-9_\-\.\x80-\xBF\xC2-\xF4]*(?:/[a-zA-Z0-9_\-\.\xC2-\xF4][a-zA-Z0-9_\-\.\x80-\xBF\xC2-\xF4]*)*)\.(?:' + extension_pattern + b')'

def extract_paths(binary_data):
    """Extract file paths from a binary string using regex."""
    binary_data = binary_data.replace(b'\xA4', b' ')  # Preprocess invalid bytes
    try:
        matches = re.findall(pattern, binary_data)
    except re.error as e:
        print(f"Regex error: {e}")
        return []
#    return [match.decode('utf-8') for match in matches if match.decode('utf-8', errors='ignore')]
#    eliminate redundant "decode" call by using intermediate variable "m_d", and loop over single calculated element 
    return [m_d for match in matches for m_d in (match.decode('utf-8', errors='ignore'),) if m_d]

def get_current_notebook_path():
    """Get the most recent notebook path from RecentFiles.plist."""
    plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
    with open(plist_path, 'rb') as f:
        return extract_paths(plistlib.load(f)[0])[-1]
    

# Test the functions
if __name__ == "__main__":
    print(f"{get_current_notebook_path() = }\n")
    
    # Example usage
    print("Example for 'extract_paths':")
    binary_data = b"data /Users/test/file.txt\xA4invalid/home/\xe6\x96\x87\xe6\xa1\xa3/doc.pdf PythonistalabTest.ipynb"
    print(extract_paths(binary_data))
    
    # Full plist contents
    print("\nFile paths in RecentFiles.plist:")
    plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
    with open(plist_path, 'rb') as f:
        # plist_data is a list of binary strings (not dictionaries)
        for binary_string in plistlib.load(f):
            print(extract_paths(binary_string))

get_current_notebook_path() = '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb'

Example for 'extract_paths':
['PythonistalabTest.ipynb']

File paths in RecentFiles.plist:
['PythonistalabTest.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb']
['get_path.py', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/get_path.py']
['pandoc.py', '/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Documents/Documents/site-packages/jupytext/pandoc.py']
['Trackpad.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/Trackpad.ipynb']
['Interfaces.ipynb', 'Interfaces.ipynb']
['Sync2way.ipynb', '/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Document

In [ ]:
import os
os.environ

In [ ]:
for i in range(len(plist_data)):
  print(f'({plist_data[i][:4] = }\n {plist_data[i][4]+plist_data[i][5]*256 =}\n {len(plist_data[i])=}\n{','.join(str(c) for c in plist_data[i][6:40])=} ) ')

(plist_data[i][:4] = b'book'
 plist_data[i][4]+plist_data[i][5]*256 =1956
 len(plist_data[i])=1956
','.join(str(c) for c in plist_data[i][6:40])='0,0,0,0,4,16,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0' ) 
(plist_data[i][:4] = b'book'
 plist_data[i][4]+plist_data[i][5]*256 =1932
 len(plist_data[i])=1932
','.join(str(c) for c in plist_data[i][6:40])='0,0,0,0,4,16,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0' ) 
(plist_data[i][:4] = b'book'
 plist_data[i][4]+plist_data[i][5]*256 =2092
 len(plist_data[i])=2092
','.join(str(c) for c in plist_data[i][6:40])='0,0,0,0,4,16,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0' ) 
(plist_data[i][:4] = b'book'
 plist_data[i][4]+plist_data[i][5]*256 =1940
 len(plist_data[i])=1940
','.join(str(c) for c in plist_data[i][6:40])='0,0,0,0,4,16,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0' ) 
(plist_data[i][:4] = b'book'
 plist_data[i][4]+plist_data[i][5]*256 =1944
 len(plist_data[i])=1944
','.join(str(c) for c

In [ ]:
plist_data[0][40:80]


b'\x00\x00\x00\x00\x00\x00\x00\x00\xac\x06\x00\x00\x07\x00\x00\x00\x01\x01\x00\x00private\x00\x03\x00\x00\x00\x01\x01\x00\x00var\x00'

In [ ]:
import plistlib

# Path to the .plist file
#plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-48E9-9F5A-7B9D5A2E0940/Library/Application Support/your_plist_file.plist"

try:
    # Read the .plist file using plistlib
    with open(plist_path, 'rb') as plist_file:
        plist_data = plistlib.load(plist_file)
    
    if not plist_data:
        print("No recent files found.")
        exit()
    
    # Parse the first binary string (most recent file)
    binary_string = plist_data[0]['book']
    
    # Validate the header
    if binary_string[:4] != b'book':
        print("Unexpected format: First 4 bytes are not 'book'.")
        exit()
    
    # Extract the length (bytes 4-5, little-endian)
    record_length = int.from_bytes(binary_string[4:6], 'little')
    print(f"Record length: {record_length} bytes")
    
    # Parse the header
    header_size = 38
    if len(binary_string) < header_size:
        print("Binary string too short to contain a header.")
        exit()
    
    # Extract the first offset (bytes 11-18, little-endian)
    next_offset = int.from_bytes(binary_string[11:19], 'little')
    print(f"First offset from header: {next_offset}")
    
    # Parse the fields
    pos = next_offset
    fields = []
    while pos < len(binary_string):
        # Check for the type indicator (e.g., 0,0,0,4,16)
        if pos + 5 > len(binary_string):
            break
        type_indicator = binary_string[pos:pos+5]
        pos += 5
        
        # Extract the offset to the next field (8 bytes, little-endian)
        if pos + 8 > len(binary_string):
            break
        next_field_offset = int.from_bytes(binary_string[pos:pos+8], 'little')
        pos += 8
        
        # Extract the field data (from current position to next_field_offset)
        if next_field_offset < pos or next_field_offset > len(binary_string):
            break
        field_data = binary_string[pos:next_field_offset]
        
        # Try to decode the field as ASCII if possible
        try:
            field_str = field_data.decode('ascii')
            print(f"Field at offset {pos}: {field_str}")
            fields.append((field_str, pos))
        except UnicodeDecodeError:
            print(f"Field at offset {pos}: (binary data, length {len(field_data)} bytes)")
            fields.append((field_data, pos))
        
        pos = next_field_offset
    
    # Look for the standalone filename and metadata list
    filename = None
    metadata = None
    full_path = None
    for field, offset in fields:
        if isinstance(field, str):
            if field.endswith(('.ipynb', '.py')) and ';' not in field:
                filename = field
                print(f"Standalone filename at offset {offset}: {filename}")
            elif field.endswith(('.ipynb', '.py')) and ';' in field:
                metadata = field
                fields_split = metadata.split(';')
                if fields_split and fields_split[-1].endswith(('.ipynb', '.py')):
                    full_path = fields_split[-1]
                    print(f"Metadata list at offset {offset}: {metadata}")
                    print(f"Full path: {full_path}")

except Exception as e:
    print(f"Failed to deserialize .plist: {e}")

Failed to deserialize .plist: byte indices must be integers or slices, not str


In [ ]:
for i,a in enumerate(plist_data[1]):
  if i%64==0: print()
  c=chr(a)
  print(c if c.isascii() and a>=32 else '.',end='',sep='')
  
print()



book........0...............................................priv
ate.........var.........mobile..........Containers..........Data
........Application.$.......77881549-3FA6-4E4B-803F-D53B172FC865
........Documents...........Notebooks...........get_path.py.(...
............ ...0...D...P...d...................................
....................-................B...............I..........
.....v...............w...............c................H.....(...
....................,...<...L...\...l...|...........A.a.........
....................................file:///private/var/mobile/.
........User........................A...c\..$.......61706673-757
5-6964-0040-766F6C756D01........................................
/private/var/mobile.........file:///private/var/........A.hT.Di{
$.......61706673-7575-6964-0040-766F6C756D06......../private/var
`.................... ........... ........... ..D........ ......
..... ........... ..........  ..h...............file:///........
System............'.@...

In [ ]:
fn=[None]*2
fn[0]='eccf58258e501d971934c668b61ff90fa17b7f29a9a81abad12096dd00790744;00;00000000;00000000;00000000;0000000000000020;com.apple.app-sandbox.read-write;01;01000004;0000000006482a8c;12;/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb'
fn[1]='afa7edfad4d2ca243dc112b100235b6df391c5342db82a19a696bc783a2f7178;00;00000000;00000000;00000000;0000000000000020;com.apple.app-sandbox.read-write;01;01000004;00000000064800be;12;/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/get_path.py'
[len(f) for f in fn]

[305, 293]

In [ ]:
pos=plist_data[0].find(bytes((305-256+1,1))) 
plist_data[0][pos-10:pos+10] 

b'did=76475\x002\x01\x00\x00\x01\x02\x00\x00ec'

In [ ]:
pos=plist_data[1].find(bytes((293-256+1,1))) 
plist_data[1][pos-10:pos+10] 

b'did=76390\x00&\x01\x00\x00\x01\x02\x00\x00af'

In [ ]:
pos=plist_data[1].find(b'private')
print(pos)
plist_data[1][pos-8:pos+15] 

60


b'\x07\x00\x00\x00\x01\x01\x00\x00private\x00\x03\x00\x00\x00\x01\x01\x00'

In [ ]:
def decodea(record):
  pos=0
  n=0
  fields=[]
  while pos<len(record):
    try:
      length=record[pos]+256*(record[pos+1]+256*(record[pos+2]+256*record[pos+3]))
      print(length)
      pos+=4
      flags=record[pos:pos+4]
      pos+=4
      content=record[pos:pos+length]
      fields.append(dict(pos=pos,length=length,flags=flags,content=content))
      pos+=length+2
    except Exception as e:
      print(pos)
      print(e)
  return fields

def decode(record):
  result={}
  pos=0
  result["recordtype"]=record[pos:pos+4]
  pos+=4
  result["recordlength"]=record[pos]+256*(record[pos+1]+256*(record[pos+2]+256*record[pos+3]))
  pos+=4
  result["flags"]=record[pos:pos+4]
  pos+=4
  fields=[]
  result["fields"]=fields
  n=0
  while pos<len(record):
    try:
      length=record[pos]+256*(record[pos+1]+256*(record[pos+2]+256*record[pos+3]))
      print(length)
      pos+=4
      flags=record[pos:pos+4]
      pos+=4
      content=record[pos:pos+length]
      fields.append(dict(pos=pos,length=length,flags=flags,content=content))
      pos+=length
    except Exception as e:
      print(pos)
      print(e)
  return result 
i=0
pos=plist_data[i].find(b'Containers')
decodea(plist_data[i][pos-8:])

10
4
16842752


[{'pos': 8,
  'length': 10,
  'flags': b'\x01\x01\x00\x00',
  'content': b'Containers'},
 {'pos': 28, 'length': 4, 'flags': b'\x01\x01\x00\x00', 'content': b'Data'},
 {'pos': 42,
  'length': 16842752,
  'flags': b'\x00\x00Ap',
  'content': b"plication\x00$\x00\x00\x00\x01\x01\x00\x0077881549-3FA6-4E4B-803F-D53B172FC865\t\x00\x00\x00\x01\x01\x00\x00Documents\x00\x00\x00\t\x00\x00\x00\x01\x01\x00\x00Notebooks\x00\x00\x00\x17\x00\x00\x00\x01\x01\x00\x00PythonistalabTest.ipynb\x00(\x00\x00\x00\x01\x06\x00\x00\x04\x00\x00\x00\x14\x00\x00\x00 \x00\x00\x000\x00\x00\x00D\x00\x00\x00P\x00\x00\x00d\x00\x00\x00\x90\x00\x00\x00\xa4\x00\x00\x00\xb8\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\xd6\x17\x06\x00\xff\xff\xff\x0f\x08\x00\x00\x00\x04\x03\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00-\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\xedB\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\x02I\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\x

In [ ]:
print(list(plist_data[0][-80:-40]))
print(len(plist_data[0]))
print(4*256+252) 
print(4*256+216) 
print(5*256+112) 


[72, 2, 0, 0, 0, 0, 0, 0, 18, 32, 0, 0, 40, 2, 0, 0, 0, 0, 0, 0, 19, 32, 0, 0, 56, 2, 0, 0, 0, 0, 0, 0, 32, 32, 0, 0, 116, 2, 0, 0]
1956
1276
1240
1392


In [ ]:
i=0

In [ ]:
i+=1
i

4